# Sentiment Analysis of Movie Reviews using a Pretrained LLM

In this notebook, I fine-tune a pretrained LLM, DistilBERT base model (uncased), on the IMDB movie-review dataset. I deploy the fine-tuned model to Hugging Face.

## Installs and Imports

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install evaluate
!pip install -U accelerate

In [3]:
import numpy as np
import torch

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import accelerate
import evaluate

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load the Data

In [5]:
ds_movies = load_dataset('imdb')

In [6]:
# Select a subset of movie reviews on which to investigate hyperparameters.
from numpy import random

rng = random.default_rng()
n_samples = 1_000
indices = rng.choice(len(ds_movies['train']), size=n_samples, replace=False)
ds_movies['train'] = ds_movies['train'].select(indices)

## Pretrained Model and Tokenizer

In [ ]:
model_name = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
def tokenize(dataset):
    """Tokenize the movie reviews of dataset."""
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_ds_movies = ds_movies.map(tokenize, batched=True)

## Train and Test

In [10]:
f1_score = evaluate.load('f1')

def compute_metrics(eval_pred):
    """Compute metrics for evaluation."""
    # raw outputs, actual labels
    logits, labels = eval_pred

    # predictions is the highest output probability
    predictions = np.argmax(logits, axis=-1)

    f1 = f1_score.compute(predictions=predictions, references=labels)
    return {'f1': f1['f1']}

In [11]:
training_args = TrainingArguments(
    output_dir='movie-review-classifier',
    eval_strategy='epoch',
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    push_to_hub=True,
    hub_model_id='derek-harnett/movie-review-classifier',
    hub_strategy='end'
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_movies['train'],
    eval_dataset=tokenized_ds_movies['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [14]:
trainer.evaluate()

{'eval_loss': 0.3089124858379364,
 'eval_f1': 0.8748628327575696,
 'eval_runtime': 466.9509,
 'eval_samples_per_second': 53.539,
 'eval_steps_per_second': 3.347,
 'epoch': 2.0}

## Push to Hugging Face

In [16]:
# trainer.push_to_hub(commit_message='test push_to_hub()')